# Apache Kafka Fundamentals

Apache Kafka is a distributed event streaming platform designed for high-throughput, fault-tolerant, and scalable real-time data pipelines. Originally developed at LinkedIn, Kafka has become the de facto standard for building streaming data architectures.

---

## Table of Contents
1. [Kafka Architecture](#kafka-architecture)
2. [Producers](#producers)
3. [Consumers](#consumers)
4. [Consumer Groups & Offset Management](#consumer-groups--offset-management)
5. [Python Code Examples](#python-code-examples)
6. [Exactly-Once Semantics](#exactly-once-semantics)
7. [Takeaways](#takeaways)

## Kafka Architecture

Kafka's architecture is built around three core components: **Brokers**, **Topics**, and **Partitions**.

### Brokers

A **broker** is a Kafka server that stores data and serves client requests. Key characteristics:

- Each broker is identified by a unique integer ID
- Brokers form a **cluster** for fault tolerance and scalability
- One broker acts as the **controller** (manages partition leadership)
- Brokers are stateless; they rely on ZooKeeper (or KRaft in newer versions) for metadata

```
┌─────────────────────────────────────────────────────────────┐
│                      Kafka Cluster                         │
│  ┌─────────────┐  ┌─────────────┐  ┌─────────────┐         │
│  │  Broker 1   │  │  Broker 2   │  │  Broker 3   │         │
│  │ (Controller)│  │             │  │             │         │
│  │             │  │             │  │             │         │
│  │ Partition 0 │  │ Partition 1 │  │ Partition 2 │         │
│  │  (Leader)   │  │  (Leader)   │  │  (Leader)   │         │
│  │ Partition 1 │  │ Partition 2 │  │ Partition 0 │         │
│  │  (Replica)  │  │  (Replica)  │  │  (Replica)  │         │
│  └─────────────┘  └─────────────┘  └─────────────┘         │
└─────────────────────────────────────────────────────────────┘
```

### Topics

A **topic** is a logical category or feed name to which records are published:

- Topics are identified by unique names (e.g., `orders`, `user-events`)
- Topics can have multiple **producers** and **consumers**
- Data in topics is retained for a configurable period (default: 7 days)
- Topics are **append-only** logs

### Partitions

Each topic is divided into **partitions** for parallelism and scalability:

- Partitions are ordered, immutable sequences of records
- Each record within a partition has a unique sequential **offset**
- Partitions enable horizontal scaling (more partitions = more parallelism)
- **Partition key** determines which partition a record goes to
- Each partition has one **leader** and zero or more **replicas**

```
Topic: orders (3 partitions)
┌─────────────────────────────────────────────────────────────┐
│ Partition 0: [0|1|2|3|4|5|6|7|8|9|...]  → Offset increases  │
│ Partition 1: [0|1|2|3|4|5|...]          → Offset increases  │
│ Partition 2: [0|1|2|3|4|5|6|7|...]      → Offset increases  │
└─────────────────────────────────────────────────────────────┘
```

### Replication Factor

Kafka replicates partitions across multiple brokers for fault tolerance:

- **Replication factor** = number of copies of each partition
- One replica is the **leader** (handles all reads/writes)
- Other replicas are **followers** (sync from leader)
- **ISR (In-Sync Replicas)**: replicas that are fully caught up with the leader

## Producers

**Producers** are client applications that publish (write) records to Kafka topics.

### Key Producer Concepts

| Concept | Description |
|---------|-------------|
| **Partitioner** | Determines which partition receives each record |
| **Batching** | Groups multiple records for efficient network usage |
| **Compression** | Reduces message size (gzip, snappy, lz4, zstd) |
| **Acknowledgments (acks)** | Controls durability guarantees |
| **Retries** | Automatic retry on transient failures |

### Acknowledgment Levels

```
acks=0: Fire and forget (fastest, least durable)
        Producer ──────► Broker (no response waited)

acks=1: Leader acknowledgment (balanced)
        Producer ──────► Leader ──────► Producer (after leader writes)

acks=all (-1): Full ISR acknowledgment (slowest, most durable)
        Producer ──────► Leader ──────► Replicas ──────► Producer
```

### Partitioning Strategies

1. **Key-based partitioning**: `hash(key) % num_partitions`
   - Records with same key always go to same partition
   - Guarantees ordering for a given key

2. **Round-robin**: When no key is specified
   - Distributes load evenly across partitions
   - No ordering guarantees

3. **Custom partitioner**: Implement your own logic

## Consumers

**Consumers** are client applications that subscribe to topics and process the records.

### Consumer Pull Model

Kafka uses a **pull-based** model where consumers request data from brokers:

- Consumers control the rate of consumption
- Enables backpressure handling
- Consumers can rewind and re-read data

### Consumer Configuration

| Configuration | Description | Typical Value |
|---------------|-------------|---------------|
| `fetch.min.bytes` | Minimum data to fetch | 1 byte |
| `fetch.max.wait.ms` | Max wait time if min bytes not available | 500ms |
| `max.partition.fetch.bytes` | Max data per partition per fetch | 1MB |
| `auto.offset.reset` | Where to start if no offset exists | `earliest` or `latest` |
| `enable.auto.commit` | Automatically commit offsets | `true` |
| `auto.commit.interval.ms` | Frequency of auto-commit | 5000ms |

## Consumer Groups & Offset Management

### Consumer Groups

A **consumer group** is a set of consumers that cooperatively consume from topics:

- Each partition is consumed by exactly **one consumer** in the group
- Enables parallel processing and horizontal scaling
- Adding consumers triggers **rebalancing**

```
Topic: orders (4 partitions)

Consumer Group A (3 consumers):
┌─────────────┐  ┌─────────────┐  ┌─────────────┐
│ Consumer 1  │  │ Consumer 2  │  │ Consumer 3  │
│ Partition 0 │  │ Partition 1 │  │ Partition 2 │
│             │  │             │  │ Partition 3 │
└─────────────┘  └─────────────┘  └─────────────┘

Consumer Group B (2 consumers) - Independent:
┌─────────────┐  ┌─────────────┐
│ Consumer 1  │  │ Consumer 2  │
│ Partition 0 │  │ Partition 2 │
│ Partition 1 │  │ Partition 3 │
└─────────────┘  └─────────────┘
```

### Offset Management

**Offsets** track the position of each consumer in each partition:

- Stored in a special topic: `__consumer_offsets`
- **Committed offset**: Last processed position acknowledged to Kafka
- **Current position**: Where the consumer is reading

```
Partition: [0|1|2|3|4|5|6|7|8|9|10|11|12|...]
                    ▲         ▲
                    │         │
             Committed    Current
              Offset      Position
```

### Offset Commit Strategies

| Strategy | Pros | Cons |
|----------|------|------|
| **Auto-commit** | Simple, no code needed | May lose/duplicate messages on failure |
| **Sync commit** | Precise control | Blocks processing |
| **Async commit** | Non-blocking | May have gaps on failure |
| **Commit specific offset** | Fine-grained control | Most complex |

### Rebalancing

Rebalancing occurs when:
- Consumer joins or leaves the group
- Consumer crashes or times out
- Partitions are added to subscribed topics

**Partition Assignment Strategies:**
- `RangeAssignor`: Assigns ranges of partitions to consumers
- `RoundRobinAssignor`: Round-robin distribution
- `StickyAssignor`: Minimizes partition movement during rebalance
- `CooperativeStickyAssignor`: Incremental rebalancing (less disruption)

## Python Code Examples

Using the `kafka-python` library for interacting with Apache Kafka.

In [ ]:
# Installation
# pip install kafka-python

from kafka import KafkaProducer, KafkaConsumer, TopicPartition
from kafka.admin import KafkaAdminClient, NewTopic
from kafka.errors import KafkaError
import json
import time

### Topic Administration

In [ ]:
def create_topic(topic_name: str, num_partitions: int = 3, replication_factor: int = 1):
    """
    Create a Kafka topic with specified configuration.
    
    Args:
        topic_name: Name of the topic to create
        num_partitions: Number of partitions for parallel processing
        replication_factor: Number of replicas for fault tolerance
    """
    admin_client = KafkaAdminClient(
        bootstrap_servers=['localhost:9092'],
        client_id='topic-admin'
    )
    
    topic = NewTopic(
        name=topic_name,
        num_partitions=num_partitions,
        replication_factor=replication_factor,
        topic_configs={
            'retention.ms': '604800000',  # 7 days
            'cleanup.policy': 'delete'
        }
    )
    
    try:
        admin_client.create_topics(new_topics=[topic], validate_only=False)
        print(f"Topic '{topic_name}' created successfully")
    except Exception as e:
        print(f"Error creating topic: {e}")
    finally:
        admin_client.close()

# Example usage
# create_topic('orders', num_partitions=6, replication_factor=3)

### Basic Producer

In [ ]:
def create_producer():
    """
    Create a Kafka producer with JSON serialization.
    
    Returns:
        KafkaProducer: Configured producer instance
    """
    producer = KafkaProducer(
        bootstrap_servers=['localhost:9092'],
        
        # Serialization
        key_serializer=lambda k: k.encode('utf-8') if k else None,
        value_serializer=lambda v: json.dumps(v).encode('utf-8'),
        
        # Durability
        acks='all',  # Wait for all ISR replicas
        
        # Reliability
        retries=3,
        retry_backoff_ms=100,
        
        # Performance
        batch_size=16384,  # 16KB batch
        linger_ms=10,  # Wait up to 10ms to batch
        compression_type='gzip',
        
        # Idempotence for exactly-once
        enable_idempotence=True
    )
    return producer


def send_message(producer: KafkaProducer, topic: str, key: str, value: dict):
    """
    Send a message to Kafka with callbacks.
    
    Args:
        producer: KafkaProducer instance
        topic: Target topic name
        key: Message key (determines partition)
        value: Message payload as dictionary
    """
    def on_success(metadata):
        print(f"Sent to {metadata.topic}:{metadata.partition} @ offset {metadata.offset}")
    
    def on_error(exception):
        print(f"Error sending message: {exception}")
    
    future = producer.send(
        topic,
        key=key,
        value=value,
        headers=[('source', b'python-client')]  # Optional headers
    )
    
    # Add callbacks
    future.add_callback(on_success)
    future.add_errback(on_error)
    
    return future


# Example usage
# producer = create_producer()
# 
# orders = [
#     {'order_id': 'ORD001', 'customer': 'Alice', 'amount': 150.00},
#     {'order_id': 'ORD002', 'customer': 'Bob', 'amount': 275.50},
#     {'order_id': 'ORD003', 'customer': 'Charlie', 'amount': 89.99},
# ]
# 
# for order in orders:
#     send_message(producer, 'orders', order['customer'], order)
# 
# producer.flush()  # Ensure all messages are sent
# producer.close()

### Basic Consumer

In [ ]:
def create_consumer(group_id: str, topics: list):
    """
    Create a Kafka consumer with JSON deserialization.
    
    Args:
        group_id: Consumer group identifier
        topics: List of topics to subscribe to
        
    Returns:
        KafkaConsumer: Configured consumer instance
    """
    consumer = KafkaConsumer(
        *topics,
        bootstrap_servers=['localhost:9092'],
        group_id=group_id,
        
        # Deserialization
        key_deserializer=lambda k: k.decode('utf-8') if k else None,
        value_deserializer=lambda v: json.loads(v.decode('utf-8')),
        
        # Offset behavior
        auto_offset_reset='earliest',  # Start from beginning if no offset
        enable_auto_commit=False,  # Manual commit for control
        
        # Performance
        fetch_min_bytes=1,
        fetch_max_wait_ms=500,
        max_poll_records=500,
        
        # Session management
        session_timeout_ms=30000,
        heartbeat_interval_ms=10000
    )
    return consumer


def consume_messages(consumer: KafkaConsumer, process_fn: callable):
    """
    Consume and process messages with manual offset commit.
    
    Args:
        consumer: KafkaConsumer instance
        process_fn: Function to process each message
    """
    try:
        while True:
            # Poll for messages (returns dict of TopicPartition -> list of records)
            records = consumer.poll(timeout_ms=1000)
            
            for topic_partition, messages in records.items():
                for message in messages:
                    print(f"Received: {message.topic}:{message.partition} @ {message.offset}")
                    print(f"  Key: {message.key}")
                    print(f"  Value: {message.value}")
                    print(f"  Timestamp: {message.timestamp}")
                    
                    # Process the message
                    process_fn(message)
            
            # Commit offsets after processing batch
            if records:
                consumer.commit()
                
    except KeyboardInterrupt:
        print("Shutting down consumer...")
    finally:
        consumer.close()


# Example usage
# def process_order(message):
#     order = message.value
#     print(f"Processing order {order['order_id']} for ${order['amount']}")
#
# consumer = create_consumer('order-processors', ['orders'])
# consume_messages(consumer, process_order)

### Manual Offset Management

In [ ]:
def consume_with_manual_offset_control(consumer: KafkaConsumer):
    """
    Demonstrates fine-grained offset management.
    
    Args:
        consumer: KafkaConsumer instance
    """
    try:
        while True:
            records = consumer.poll(timeout_ms=1000)
            
            for topic_partition, messages in records.items():
                for message in messages:
                    try:
                        # Process message
                        process_message(message)
                        
                        # Commit specific offset (next offset to read)
                        consumer.commit({
                            topic_partition: message.offset + 1
                        })
                        
                    except Exception as e:
                        print(f"Error processing message: {e}")
                        # Don't commit - message will be reprocessed
                        
    except KeyboardInterrupt:
        pass
    finally:
        consumer.close()


def seek_to_specific_offset(consumer: KafkaConsumer, topic: str, partition: int, offset: int):
    """
    Seek to a specific offset in a partition.
    Useful for replaying messages or skipping ahead.
    
    Args:
        consumer: KafkaConsumer instance
        topic: Topic name
        partition: Partition number
        offset: Target offset
    """
    tp = TopicPartition(topic, partition)
    
    # Assign specific partition (bypasses consumer group)
    consumer.assign([tp])
    
    # Seek to specific offset
    consumer.seek(tp, offset)
    
    print(f"Seeked to {topic}:{partition} @ offset {offset}")


def seek_to_timestamp(consumer: KafkaConsumer, topic: str, partition: int, timestamp_ms: int):
    """
    Seek to a specific timestamp in a partition.
    
    Args:
        consumer: KafkaConsumer instance
        topic: Topic name
        partition: Partition number
        timestamp_ms: Target timestamp in milliseconds
    """
    tp = TopicPartition(topic, partition)
    consumer.assign([tp])
    
    # Get offset for timestamp
    offsets = consumer.offsets_for_times({tp: timestamp_ms})
    
    if offsets[tp]:
        consumer.seek(tp, offsets[tp].offset)
        print(f"Seeked to offset {offsets[tp].offset} for timestamp {timestamp_ms}")
    else:
        print(f"No offset found for timestamp {timestamp_ms}")


def process_message(message):
    """Placeholder for message processing logic."""
    print(f"Processing: {message.value}")

### Consumer Group Pattern with Graceful Shutdown

In [ ]:
import signal
import threading
from typing import Callable, Optional


class KafkaConsumerWorker:
    """
    A robust Kafka consumer with graceful shutdown support.
    """
    
    def __init__(
        self,
        topics: list,
        group_id: str,
        bootstrap_servers: list = ['localhost:9092'],
        process_fn: Optional[Callable] = None
    ):
        self.topics = topics
        self.group_id = group_id
        self.bootstrap_servers = bootstrap_servers
        self.process_fn = process_fn or self._default_processor
        self._shutdown = threading.Event()
        self._consumer: Optional[KafkaConsumer] = None
        
    def _default_processor(self, message):
        """Default message processor."""
        print(f"Message: {message.value}")
    
    def _create_consumer(self) -> KafkaConsumer:
        """Create and configure the consumer."""
        return KafkaConsumer(
            *self.topics,
            bootstrap_servers=self.bootstrap_servers,
            group_id=self.group_id,
            key_deserializer=lambda k: k.decode('utf-8') if k else None,
            value_deserializer=lambda v: json.loads(v.decode('utf-8')),
            auto_offset_reset='earliest',
            enable_auto_commit=False,
            max_poll_records=100,
            session_timeout_ms=30000,
            heartbeat_interval_ms=10000
        )
    
    def start(self):
        """Start consuming messages."""
        self._consumer = self._create_consumer()
        print(f"Consumer started for topics: {self.topics}")
        
        try:
            while not self._shutdown.is_set():
                records = self._consumer.poll(timeout_ms=1000)
                
                for topic_partition, messages in records.items():
                    for message in messages:
                        if self._shutdown.is_set():
                            break
                        
                        try:
                            self.process_fn(message)
                        except Exception as e:
                            print(f"Error processing message: {e}")
                            # Could implement dead letter queue here
                
                if records:
                    self._consumer.commit()
                    
        finally:
            self._cleanup()
    
    def shutdown(self):
        """Signal graceful shutdown."""
        print("Shutdown signal received...")
        self._shutdown.set()
    
    def _cleanup(self):
        """Clean up resources."""
        if self._consumer:
            try:
                self._consumer.commit()  # Final commit
            except Exception:
                pass
            self._consumer.close()
            print("Consumer closed gracefully")


# Example usage with signal handling
# def order_processor(message):
#     order = message.value
#     print(f"Processing order: {order}")
#     time.sleep(0.1)  # Simulate processing
#
# worker = KafkaConsumerWorker(
#     topics=['orders'],
#     group_id='order-service',
#     process_fn=order_processor
# )
#
# # Handle SIGINT/SIGTERM
# signal.signal(signal.SIGINT, lambda sig, frame: worker.shutdown())
# signal.signal(signal.SIGTERM, lambda sig, frame: worker.shutdown())
#
# worker.start()

## Exactly-Once Semantics

Kafka provides **exactly-once semantics (EOS)** to ensure each record is processed exactly once, even in the presence of failures.

### Delivery Guarantees

| Guarantee | Description | Potential Issues |
|-----------|-------------|------------------|
| **At-most-once** | Messages may be lost, never duplicated | Data loss |
| **At-least-once** | Messages never lost, may be duplicated | Duplicate processing |
| **Exactly-once** | Messages processed exactly once | Complex implementation |

### Components of Exactly-Once

1. **Idempotent Producer** (`enable.idempotence=true`)
   - Producer assigns sequence numbers to messages
   - Broker detects and deduplicates retries
   - Guarantees exactly-once delivery to a single partition

2. **Transactional Producer**
   - Atomic writes across multiple partitions
   - All-or-nothing semantics
   - Requires `transactional.id`

3. **Transactional Consumer** (`isolation.level=read_committed`)
   - Only reads committed messages
   - Skips aborted transactions

### Transaction Flow

```
┌─────────────────────────────────────────────────────────────┐
│                    Transaction Lifecycle                    │
├─────────────────────────────────────────────────────────────┤
│  1. initTransactions()    → Register transactional.id      │
│  2. beginTransaction()    → Start atomic unit              │
│  3. send()                → Buffer messages                │
│  4. sendOffsetsToTxn()    → Include consumer offsets       │
│  5. commitTransaction()   → Atomically commit all          │
│     OR abortTransaction() → Rollback everything            │
└─────────────────────────────────────────────────────────────┘
```

In [ ]:
def create_transactional_producer(transactional_id: str):
    """
    Create a transactional producer for exactly-once semantics.
    
    Args:
        transactional_id: Unique identifier for this producer's transactions
        
    Returns:
        KafkaProducer: Transactional producer instance
    """
    producer = KafkaProducer(
        bootstrap_servers=['localhost:9092'],
        key_serializer=lambda k: k.encode('utf-8') if k else None,
        value_serializer=lambda v: json.dumps(v).encode('utf-8'),
        
        # Exactly-once configuration
        enable_idempotence=True,
        transactional_id=transactional_id,
        
        # Required for transactions
        acks='all',
        retries=2147483647,  # Max retries
        max_in_flight_requests_per_connection=5
    )
    
    # Initialize transactions (must be called once)
    producer.init_transactions()
    
    return producer


def send_with_transaction(producer: KafkaProducer, messages: list):
    """
    Send multiple messages atomically in a transaction.
    
    Args:
        producer: Transactional KafkaProducer
        messages: List of (topic, key, value) tuples
    """
    try:
        producer.begin_transaction()
        
        for topic, key, value in messages:
            producer.send(topic, key=key, value=value)
        
        producer.commit_transaction()
        print(f"Transaction committed: {len(messages)} messages")
        
    except KafkaError as e:
        print(f"Transaction failed: {e}")
        producer.abort_transaction()


# Example: Atomic multi-topic write
# producer = create_transactional_producer('order-processor-1')
#
# messages = [
#     ('orders', 'order-123', {'order_id': '123', 'status': 'completed'}),
#     ('inventory', 'sku-456', {'sku': '456', 'quantity': -1}),
#     ('notifications', 'user-789', {'user_id': '789', 'message': 'Order shipped'})
# ]
#
# send_with_transaction(producer, messages)

### Read-Process-Write Pattern (Consume-Transform-Produce)

In [ ]:
from kafka import OffsetAndMetadata


def exactly_once_consume_transform_produce(
    consumer: KafkaConsumer,
    producer: KafkaProducer,
    transform_fn: Callable,
    output_topic: str
):
    """
    Implements exactly-once consume-transform-produce pattern.
    
    This pattern ensures:
    - Input is consumed exactly once
    - Output is produced exactly once
    - Consumer offsets and output are committed atomically
    
    Args:
        consumer: KafkaConsumer (with enable_auto_commit=False)
        producer: Transactional KafkaProducer
        transform_fn: Function to transform input message to output
        output_topic: Topic to write transformed messages
    """
    try:
        while True:
            records = consumer.poll(timeout_ms=1000)
            
            if not records:
                continue
            
            try:
                # Start transaction
                producer.begin_transaction()
                
                offsets_to_commit = {}
                
                for topic_partition, messages in records.items():
                    for message in messages:
                        # Transform and produce
                        output_value = transform_fn(message.value)
                        producer.send(
                            output_topic,
                            key=message.key,
                            value=output_value
                        )
                        
                        # Track offsets
                        offsets_to_commit[topic_partition] = OffsetAndMetadata(
                            message.offset + 1, None
                        )
                
                # Commit offsets as part of transaction
                producer.send_offsets_to_transaction(
                    offsets_to_commit,
                    consumer.group_id
                )
                
                # Commit transaction atomically
                producer.commit_transaction()
                print(f"Transaction committed: {len(offsets_to_commit)} partitions")
                
            except KafkaError as e:
                print(f"Transaction failed, aborting: {e}")
                producer.abort_transaction()
                # Consumer will re-read uncommitted messages
                
    except KeyboardInterrupt:
        print("Shutting down...")
    finally:
        consumer.close()
        producer.close()


# Example: Order enrichment pipeline
# def enrich_order(order: dict) -> dict:
#     """Add processing timestamp and status."""
#     return {
#         **order,
#         'processed_at': time.time(),
#         'status': 'enriched'
#     }
#
# consumer = KafkaConsumer(
#     'raw-orders',
#     bootstrap_servers=['localhost:9092'],
#     group_id='order-enricher',
#     enable_auto_commit=False,
#     isolation_level='read_committed',  # Only read committed messages
#     value_deserializer=lambda v: json.loads(v.decode('utf-8'))
# )
#
# producer = create_transactional_producer('order-enricher-1')
#
# exactly_once_consume_transform_produce(
#     consumer, producer, enrich_order, 'enriched-orders'
# )

### Idempotent Consumer Pattern

When exactly-once semantics aren't available end-to-end, make consumers idempotent:

In [ ]:
from typing import Set


class IdempotentOrderProcessor:
    """
    Idempotent consumer that tracks processed message IDs.
    
    In production, use a persistent store (Redis, database) instead of in-memory set.
    """
    
    def __init__(self):
        # In production: use Redis SET or database table
        self._processed_ids: Set[str] = set()
    
    def _get_message_id(self, message) -> str:
        """Extract unique message identifier."""
        # Option 1: Use message offset (partition-specific)
        # return f"{message.topic}-{message.partition}-{message.offset}"
        
        # Option 2: Use business key from message value
        return message.value.get('order_id', str(message.offset))
    
    def _is_processed(self, message_id: str) -> bool:
        """Check if message was already processed."""
        return message_id in self._processed_ids
    
    def _mark_processed(self, message_id: str):
        """Mark message as processed."""
        self._processed_ids.add(message_id)
        # In production: SET with TTL or database insert
    
    def process(self, message) -> bool:
        """
        Process message idempotently.
        
        Returns:
            bool: True if processed, False if duplicate
        """
        message_id = self._get_message_id(message)
        
        if self._is_processed(message_id):
            print(f"Skipping duplicate: {message_id}")
            return False
        
        # Process the message
        order = message.value
        print(f"Processing order {order['order_id']}: ${order.get('amount', 0)}")
        
        # Simulate processing (database write, API call, etc.)
        # If this fails, message will be reprocessed but idempotency prevents duplicates
        
        self._mark_processed(message_id)
        return True


# Example usage
# processor = IdempotentOrderProcessor()
# consumer = create_consumer('order-handlers', ['orders'])
#
# for message in consumer:
#     processor.process(message)
#     consumer.commit()

## Takeaways

### Key Concepts

| Concept | Key Points |
|---------|------------|
| **Architecture** | Brokers form clusters; topics are split into partitions; partitions replicated across brokers |
| **Partitioning** | Key-based partitioning guarantees ordering per key; enables horizontal scaling |
| **Consumer Groups** | Partition-to-consumer mapping; enables parallel processing; triggers rebalancing on changes |
| **Offset Management** | Stored in `__consumer_offsets`; commit strategies affect delivery guarantees |
| **Exactly-Once** | Idempotent producers + transactions + read_committed consumers |

### Best Practices

1. **Producer Configuration**
   - Use `acks=all` for durability
   - Enable idempotence for exactly-once delivery
   - Configure appropriate batch size and linger time

2. **Consumer Configuration**
   - Disable auto-commit for precise control
   - Use `read_committed` isolation for transactional reads
   - Implement graceful shutdown to avoid rebalancing delays

3. **Partition Design**
   - Choose partition key based on access patterns
   - Plan partition count for peak throughput (hard to reduce later)
   - Consider consumer group size when planning partitions

4. **Error Handling**
   - Implement idempotent consumers for at-least-once delivery
   - Use dead letter queues for poison messages
   - Monitor consumer lag to detect processing issues

5. **Operational Excellence**
   - Set appropriate retention policies
   - Monitor ISR shrinkage as indicator of broker health
   - Use Cooperative Sticky assignor to minimize rebalance impact

### When to Use Kafka

✅ **Good Fit:**
- High-throughput event streaming
- Real-time data pipelines
- Event sourcing and CQRS
- Log aggregation
- Microservices communication

❌ **Consider Alternatives:**
- Simple request-response (use HTTP/gRPC)
- Small-scale messaging (use Redis/RabbitMQ)
- Complex routing logic (use RabbitMQ)
- Guaranteed ordering across topics (use database)